In [4]:
pip install dlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install serial

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install pyserial

In [8]:
import cv2
import dlib
import numpy as np
import serial
import pandas as pd
import time

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor(r'C:\Users\khush\Desktop\shape_predictor_68_face_landmarks.dat')
arduino = serial.Serial('COM10', 9600)  # Adjust the COM port as necessary
time.sleep(2) 

model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip
    (0.0, -330.0, -65.0),        # Chin
    (-225.0, 170.0, -135.0),     # Left eye corner
    (225.0, 170.0, -135.0),      # Right eye corner
    (-150.0, -150.0, -125.0),    # Left mouth corner
    (150.0, -150.0, -125.0)      # Right mouth corner
])

def get_camera_matrix(frame):
    height, width = frame.shape[:2]
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")
    return camera_matrix

def get_2d_landmarks(shape):
    image_points = np.array([
        (shape.part(30).x, shape.part(30).y),     # Nose tip
        (shape.part(8).x, shape.part(8).y),       # Chin
        (shape.part(36).x, shape.part(36).y),     # Left eye corner
        (shape.part(45).x, shape.part(45).y),     # Right eye corner
        (shape.part(48).x, shape.part(48).y),     # Left mouth corner
        (shape.part(54).x, shape.part(54).y)      # Right mouth corner
    ], dtype="double")
    return image_points

def estimate_head_pose(frame, landmarks):
    image_points = get_2d_landmarks(landmarks)
    camera_matrix = get_camera_matrix(frame)
    dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion

    success, rotation_vector, translation_vector = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs
    )

    # Calculate pitch, yaw, roll
    rmat, _ = cv2.Rodrigues(rotation_vector)
    pose_angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
    pitch, yaw, roll = pose_angles

    return pitch, yaw, roll


# Start capturing video
cap = cv2.VideoCapture(0)

pose_data = []  # Initialize the pose_data list
start_time = time.time()  # Start time record

while (time.time() - start_time) < 10:  # Run loop for 10 seconds
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    for face in faces:
        landmarks = landmark_predictor(gray, face)

        pitch, yaw, roll = estimate_head_pose(frame, landmarks)

        pose_data.append({'Pitch': pitch, 'Yaw': yaw, 'Roll': roll})

        servo_angle = int(yaw)   # Adjust for your servo range
        servo_angle = max(0, min(180, servo_angle))  # Constrain to valid range
        arduino.write(f"{servo_angle}\n".encode('utf-8'))

        cv2.putText(frame, f"Pitch: {pitch:.2f}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Yaw: {yaw:.2f}", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Roll: {roll:.2f}", (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Head Pose Estimation", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Ensure that pose_data has content before trying to create a DataFrame
if pose_data:
    df = pd.DataFrame(pose_data)
    df.to_csv('head_posture_data.csv', index=False)
else:
    print("No data collected during the 10 seconds.")

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)  # Ensure window closes properly


SerialException: could not open port 'COM10': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [9]:
import cv2
import dlib
import numpy as np
import serial
import pandas as pd
import time

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor(r'C:\Users\khush\Desktop\shape_predictor_68_face_landmarks.dat')
arduino = serial.Serial('COM11', 9600)  # Adjust the COM port as necessary
time.sleep(2)

model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip
    (0.0, -330.0, -65.0),        # Chin
    (-225.0, 170.0, -135.0),     # Left eye corner
    (225.0, 170.0, -135.0),      # Right eye corner
    (-150.0, -150.0, -125.0),    # Left mouth corner
    (150.0, -150.0, -125.0)      # Right mouth corner
])

def get_camera_matrix(frame):
    height, width = frame.shape[:2]
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")
    return camera_matrix

def get_2d_landmarks(shape):
    image_points = np.array([
        (shape.part(30).x, shape.part(30).y),     # Nose tip
        (shape.part(8).x, shape.part(8).y),       # Chin
        (shape.part(36).x, shape.part(36).y),     # Left eye corner
        (shape.part(45).x, shape.part(45).y),     # Right eye corner
        (shape.part(48).x, shape.part(48).y),     # Left mouth corner
        (shape.part(54).x, shape.part(54).y)      # Right mouth corner
    ], dtype="double")
    return image_points

def estimate_head_pose(frame, landmarks):
    image_points = get_2d_landmarks(landmarks)
    camera_matrix = get_camera_matrix(frame)
    dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion

    success, rotation_vector, translation_vector = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs
    )

    # Calculate pitch, yaw, roll
    rmat, _ = cv2.Rodrigues(rotation_vector)
    pose_angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
    pitch, yaw, roll = pose_angles

    return pitch, yaw, roll


# Start capturing video
cap = cv2.VideoCapture(0)

pose_data = []  # List to store pose data

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)
    print(f"Faces detected: {len(faces)}")  # Print the number of faces detected

    if len(faces) == 0:
        print("No faces detected.")
    
    for face in faces:
        landmarks = landmark_predictor(gray, face)
        if landmarks is None:
            print("Landmarks not found.")
            continue  # Skip if landmarks are not found

        pitch, yaw, roll = estimate_head_pose(frame, landmarks)

        pose_data.append({'Pitch': pitch, 'Yaw': yaw, 'Roll': roll})

        # Adjust yaw to fit the servo's range of 0-180
        servo_angle = int(yaw) + 90  # Add 90 to map yaw to the servo range
        servo_angle = max(0, min(180, servo_angle))  # Constrain to valid range
        arduino.write(f"{servo_angle}\n".encode('utf-8'))  # Send angle to Arduino

        # Display the pitch, yaw, and roll on the frame
        cv2.putText(frame, f"Pitch: {pitch:.2f}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Yaw: {yaw:.2f}", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Roll: {roll:.2f}", (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Head Pose Estimation", frame)

    # Check if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print the number of data points collected
print(f"Number of data points collected: {len(pose_data)}")

# Ensure pose_data has content before writing to CSV
if pose_data:
    print("Saving data to CSV...")
    df = pd.DataFrame(pose_data)
    df.to_csv(r'C:\Users\khush\Desktop\head_posture_data.csv', index=False)  # Using absolute path
    print("Data saved to head_posture_data.csv")
else:
    print("No pose data collected.")

cap.release()
cv2.destroyAllWindows()


Faces detected: 0
No faces detected.
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 0
No faces detected.
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 1
Faces detected: 

SerialTimeoutException: Write timeout